# 물체 탐지(Object detection) - YOLO Keras

DarkNet의 모델을 Keras로 변환하여 물체 탐지 실행.

https://github.com/qqwweee/keras-yolo3


## 프로젝트 다운로드


In [1]:
%cd /content

!rm -rf keras-yolo3

!git clone https://github.com/qqwweee/keras-yolo3


/content
Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 150.95 KiB | 408.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.


## YOLO3 모델 다운로드


In [2]:
%cd /content/keras-yolo3
!wget wget https://pjreddie.com/media/files/yolov3.weights

/content/keras-yolo3
--2019-08-07 02:16:08--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2019-08-07 02:16:08--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  23.2MB/s    in 11s     

2019-08-07 02:16:20 (21.6 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

FINISHED --2019-08-07 02:16:20--
Total wall clock time: 12s
Downloaded: 1 files, 237M in 11s (21.6 MB/s)


## 모델 파일 변환

DarkNet의 모델파일을 Keras에서 사용할 수 있는 모델 파일로 변환한다.


- convert.py : 변환 실행 파일
- yolov3.cfg : Darknet에서 사용하는 모델 구조 정의 파일
- yolov3.weight : Darknet으로 학습된 모델 파일

실행 결과로 다음의 h5 파일로 변환된다. 
- yolov3.h5 : 변환된 모델 파일


In [0]:
!head -40 yolov3.cfg 

[net]
# Testing
batch=1
subdivisions=1
# Training
# batch=64
# subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1


In [0]:
# 실행하기전 기존것이 있다면 삭제
!rm -rf yolov3.h5

# 실행 
%run convert.py yolov3.cfg yolov3.weights yolov3.h5

Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.
Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)


## 물체 탐지 실행

다음의 파일이 사용된다.
- yolov3.h5 : 모델 파일
- model_data/coco_classes.txt : 클래스 이름


In [0]:
!wget -O street_view.jpg http://beltlineorg-wpengine.netdna-ssl.com/wp-content/uploads/2012/12/IMG_5669-flickr-websized.jpg

from IPython.display import Image, display
display(Image(filename='street_view.jpg')) 

In [0]:
!cat model_data/coco_classes.txt

In [0]:
from IPython.display import display 
from PIL import Image
from yolo import YOLO

def do_object_detection(file, model_path, class_path):
  
  yolo = YOLO(model_path=model_path, classes_path=class_path)

  image = Image.open(file)
  result_image = yolo.detect_image(image)

  display(result_image)


In [0]:
do_object_detection('street_view.jpg', 'yolov3.h5', 'model_data/coco_classes.txt')



# 커스텀 데이터 학습


## 커스터 데이터 다운로드

standford dogs 데이터셋 중 10개 클래스만 추린 데이타

In [3]:
%cd /content/keras-yolo3

!rm -rf dogs.tar.gz

!wget https://github.com/dhrim/hallym_medi_workshop_2020/raw/master/material/dogs.tar.gz


/content/keras-yolo3
--2019-08-07 02:16:25--  https://github.com/dhrim/hallym_medi_workshop_2020/raw/master/material/dogs.tar.gz
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dhrim/hallym_medi_workshop_2020/raw/master/material/dogs.tar.gz [following]
--2019-08-07 02:16:26--  https://raw.githubusercontent.com/dhrim/hallym_medi_workshop_2020/raw/master/material/dogs.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62885106 (60M) [application/octet-stream]
Saving to: ‘dogs.tar.gz’

dogs.tar.gz         100%[===================>]  59.97M   158MB/s    in 0.4s    

2019-08-07 02:16:30 (158 MB/s) - ‘

In [4]:
%cd /content/keras-yolo3

!rm -rf dogs

!tar xfz dogs.tar.gz

!ls -al dogs | head -10

/content/keras-yolo3
total 128
drwxr-xr-x 12  501 staff  4096 Jul 23 15:49 .
drwxr-xr-x  7 root root   4096 Aug  7 02:16 ..
drwxr-xr-x  2  501 staff 12288 Aug  7 02:16 n02085620-Chihuahua
drwxr-xr-x  2  501 staff 12288 Aug  7 02:16 n02085782-Japanese_spaniel
drwxr-xr-x  2  501 staff 12288 Aug  7 02:16 n02085936-Maltese_dog
drwxr-xr-x  2  501 staff 12288 Aug  7 02:16 n02086079-Pekinese
drwxr-xr-x  2  501 staff 12288 Aug  7 02:16 n02086240-Shih-Tzu
drwxr-xr-x  2  501 staff 12288 Aug  7 02:16 n02086646-Blenheim_spaniel
drwxr-xr-x  2  501 staff 12288 Aug  7 02:16 n02086910-papillon


## 데이터 준비

2개의 파일

- 클래스 파일
- 레이블링 파일

그리고 이미지 파일들

In [5]:
!wget https://github.com/dhrim/hallym_medi_workshop_2020/raw/master/material/dogs_classes.txt
!wget https://github.com/dhrim/hallym_medi_workshop_2020/raw/master/material/dogs_annotations.txt
  

--2019-08-07 02:16:40--  https://github.com/dhrim/hallym_medi_workshop_2020/raw/master/material/dogs_classes.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dhrim/hallym_medi_workshop_2020/raw/master/material/dogs_classes.txt [following]
--2019-08-07 02:16:41--  https://raw.githubusercontent.com/dhrim/hallym_medi_workshop_2020/raw/master/material/dogs_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228 [text/plain]
Saving to: ‘dogs_classes.txt’

dogs_classes.txt    100%[===================>]     228  --.-KB/s    in 0s      

2019-08-07 02:16:42 (20.6 MB/s) - ‘dogs_classes.txt’ saved [

In [7]:
!cat dogs_classes.txt

n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound


In [8]:
!wc dogs_annotations.txt
!head -10 dogs_annotations.txt

  1918   3934 122615 dogs_annotations.txt
dogs/n02086240-Shih-Tzu/n02086240_8825.jpg 97,46,436,464,0
dogs/n02086240-Shih-Tzu/n02086240_4608.jpg 0,100,496,371,0
dogs/n02086240-Shih-Tzu/n02086240_1779.jpg 213,114,361,331,0
dogs/n02086240-Shih-Tzu/n02086240_5352.jpg 0,57,375,491,0
dogs/n02086240-Shih-Tzu/n02086240_9098.jpg 156,9,348,375,0
dogs/n02086240-Shih-Tzu/n02086240_963.jpg 0,8,325,492,0
dogs/n02086240-Shih-Tzu/n02086240_5541.jpg 63,105,345,284,0
dogs/n02086240-Shih-Tzu/n02086240_7100.jpg 6,23,372,456,0
dogs/n02086240-Shih-Tzu/n02086240_4629.jpg 32,57,451,373,0
dogs/n02086240-Shih-Tzu/n02086240_1958.jpg 43,16,375,495,0


## 학습 실행

학습에 필요한 파일은 3개이다.

- 클래스 파일
- 레이블링 파일
- 기반 학습 모델




그런데 2019/08/07 현재 이 파일 이름들은 train.py안에 하드코딩 되어있다.
- 클래스 파일 : model_data/voc_classes.txt
- 레이블링 파일 : train.txt
- 기반 학습 모델 : model_data/yolo_weights.h5

하드 코딩된 바 데로 이를음 변경하고 실행한다.



In [0]:
![ ! -f model_data/voc_classes.txt.org ] && mv model_data/voc_classes.txt model_data/voc_classes.txt.org
![ ! -f train.txt.org ] && mv train.txt train.txt.org
![ ! -f model_data/yolo_wieghts.org ] && mv model_data/yolo_weights.h5 model_data/yolo_wieghts.h5.org

![ ! -f model_data/voc_classes.txt ] && cp dogs_classes.txt model_data/voc_classes.txt
![ ! -f train.txt ] && cp dogs_annotations.txt train.txt
![ ! -f model_data/yolo_weights.h5 ] && cp yolov3.h5 model_data/yolo_weights.h5


mv: cannot stat 'model_data/yolo_weights.h5': No such file or directory


In [0]:
!ls -al *.txt

-rw-r--r-- 1 root root 122601 Aug  6 20:33 dogs_annotations.txt
-rw-r--r-- 1 root root    228 Aug  6 18:44 dogs_classes.txt


In [0]:
!python3 train.py

Using TensorFlow backend.
W0806 20:44:08.706888 139875781281664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0806 20:44:08.707202 139875781281664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0806 20:44:08.720846 139875781281664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 20:44:08.721091 139875781281664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.